In [1]:
import cv2
import math
import numpy as np

In [2]:
path = r"../test_images/red4.webp"
img = cv2.imread(path)
# img = cv2.bilateralFilter(img, 9,75,75)
img = cv2.GaussianBlur(img, (15, 15), cv2.BORDER_DEFAULT)

# Converting to HSV color space for better red-color segmentation
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

cv2.imshow('red line',img)
cv2.imshow('hsv',img_hsv)
cv2.waitKey(0)
cv2.destroyAllWindows()
print(f"Image dimensions: {img.shape}")

Image dimensions: (900, 900, 3)


In [3]:
# Lower and upper HSV bounds for red (at both ends of the spectrum)
lowerb1 = np.array([0, 75, 20])
upperb1 = np.array([10, 255, 255])

lowerb2 = np.array([160, 75, 20])
upperb2 = np.array([180, 255, 255])

#the above are the pixel values that specify two ranges for the color red

# Masking the path to be followed
lower_mask  = cv2.inRange(img_hsv, lowerb1, upperb1)
upper_mask  = cv2.inRange(img_hsv, lowerb2, upperb2)

mask = lower_mask + upper_mask
cv2.imshow('lower mask',lower_mask)
cv2.imshow('upper mask',upper_mask)
cv2.imshow('full masked',mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
img_edges = cv2.Canny(mask, 50, 200, None, 3)
#lines = cv2.HoughLinesP(img_edges, 1, np.pi/180, 50, maxLineGap=50)
lines = cv2.HoughLines(img_edges, 1, np.pi / 180, 150, None, 0, 0)
print(lines.shape)
# for line in lines:
#     print(line[0])
#     x1,y1,x2,y2=line[0]
#     cv2.line(img,(x1,y1),(x2,y2),(0,255,0),thickness=5)
cv2.imshow('canny',img_edges)
cv2.imshow('original image with lines',img)
cv2.waitKey(0)
cv2.destroyAllWindows()
print(f"number of lines detected is {len(lines)}")

(5, 1, 2)
number of lines detected is 5


In [17]:

line_taken = 0
rho = lines[line_taken][0][0]
theta = lines[line_taken][0][1]

theta_line = 90 - math.degrees(theta)
theta_norm_line = math.atan(- 1 / math.tan(math.radians(theta_line))) #in radians
thickness = abs(np.count_nonzero(mask[len(mask) - 15]) / math.cos(math.pi - theta_norm_line))

# Distance from center calculations 
a = math.cos(theta)
b = math.sin(theta)
x0 = a * rho
y0 = b * rho
pt1 = np.array((int(x0 + 1000*(-b)), int(y0 + 1000*(a))))
pt2 = np.array((int(x0 - 1000*(-b)), int(y0 - 1000*(a))))
center = (img.shape[0]/2, img.shape[1]/2)
distanceFromOrigin = np.cross(pt2 - pt1, pt1-center)/np.linalg.norm(pt1-pt2)
cv2.line(img,pt1,pt2,(0,255,0),thickness=3)
cv2.imshow('original image with lines',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

print(f"The number of hough lines is: {len(lines)}")
print(f"First Hough Line is: rho({lines[0][0][0]}), theta({90-math.degrees(lines[0][0][1])})")
print(f"Thicnkess (in pixels): {thickness}")
print(f"Distance from center (in pixels): {distanceFromOrigin} ")

The number of hough lines is: 5
First Hough Line is: rho(67.0), theta(-40.999995676853075)
Thicnkess (in pixels): 234.7349957274396
Distance from center (in pixels): 22.614404571462238 


In [5]:
empty_image = np.zeros((900,900,3))
cv2.line(empty_image,(50,0),(50,200),(0,255,0),4)
cv2.imshow('empty_image',empty_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
moments = cv2.moments(mask)
print(moments)
cX = int(moments['m10'] / moments['m00'])
cY = int(moments['m01'] / moments['m00'])

print(f"cY is {cY}")
print(f"cX is {cX}")
cv2.arrowedLine(img_hsv, (int(img_hsv.shape[1]/2), int(img_hsv.shape[0])), (cX, cY), (255, 0, 0), 10)
cv2.imshow('orig',img_hsv)
cv2.waitKey(0)
cv2.destroyAllWindows()
print(f'Vector to be followed is:({cX - img_hsv.shape[1]/2}, {cY - img_hsv.shape[0]})')

{'m00': 32342415.0, 'm10': 13717988415.0, 'm01': 17204119935.0, 'm20': 7782762000225.0, 'm11': 8949756831315.0, 'm02': 10601425327605.0, 'm30': 4980547962237375.0, 'm21': 5544844628758395.0, 'm12': 6249954824139915.0, 'm03': 7171318524025635.0, 'mu20': 1964297109644.8062, 'mu11': 1652654312576.045, 'mu02': 1449921250439.9392, 'mu30': 13193207088767.012, 'mu21': 2965449195973.533, 'mu12': -4839563845010.666, 'mu03': -10502344818292.703, 'nu20': 0.001877856002620039, 'nu11': 0.0015799273978914527, 'nu02': 0.001386115832526645, 'nu30': 2.2177841014842818e-06, 'nu21': 4.984933561900214e-07, 'nu12': -8.135328795620287e-07, 'nu03': -1.7654489321361941e-06}
cY is 531
cX is 424
Vector to be followed is:(-26.0, -369)
